In [5]:
!pip install pymongo

# Justification of the model with example: 

For the assignemnt I used embedded model as we have pre saved csv dataset and there would be no new data added. For quicker data read embedded model is preferred above reference model which might have been useful for requirements to have more write operations.

The dataset 'climate_historic' contains only one record is produced per day. While the dataset 'hotspot_historic' can have zero to many records on a single day. As a result, using the date as the connecting data point between to the two collections, we can model a one-to-many relationship, where a climate record, can have many hotspot records. Thus in need of extraction of information about hotspots we can query easily using embedded model as it would have all the hotspot data for a particular day in single document based on climate_historic dataset. Which for reference model case would require more queries to be made for the same result. It aslo eliminates the need for joins. Another important benefit, is that operations peformed on a embedded model is always atomic, whilst in a reference model, atomic operations can never be assumed.

A potential problem with this approach is that this can lead to large documents, especially when in this case, a 'climate' may have a large subset of 'hotspots'. However we have 4mb space provided for a single document and based on that it can server much more than 100 hotspot documenets too for a single day, which can't happen in a single day, but the capacity is there for the model to hold the data based on that. 

Example 


{'_id': ObjectId('647cc92054ffcc378202d047'),

 'air_temperature_celcius': 14.0,
 
 'date': datetime.datetime(2022, 11, 9, 0, 0),
 
 'ghi': 119.0,
 
 'hotspots': {'confidence': 100.0,
 
              'latitude': -37.477,
              
              'longitude': 143.352,
              
              'surface_temperature_celcius': 93.0,
              
              'time': datetime.datetime(2022, 11, 9, 4, 16, 48)},
              
 'max_wind_speed': 13.0,
 
 'precipitation': 0.0,
 
 'precipitation_type': 'I',
 
 'relative_humidity': 50.9,
 
 'station': 948702,
 
 'windspeed_knots': 5.9}

In [6]:
from datetime import datetime
from pprint import pprint
from pymongo import MongoClient
import pymongo
import pandas

In [7]:
# Connect to MongoClient & get/create assignment database
client = pymongo.MongoClient('192.168.1.5', 27017)
db = client.fit3182_assignment_db
collection = db.FIT_COMPLEX

# Drop existing collection, to reload data from CSV
collection.drop()

# Read data from CSV using pandas
hotspot_data = pandas.read_csv('hotspot_historic.csv')
climate_data = pandas.read_csv('climate_historic.csv')

In [8]:
print(client.list_database_names())

['admin', 'config', 'local']


In [9]:
# Connect to MongoClient & get/create assignment database
client = pymongo.MongoClient('192.168.1.5', 27017)
db = client.fit3182_assignment_db
collection = db.FIT_COMPLEX

# Drop existing collection, to reload data from CSV
collection.drop()

# Read data from CSV using pandas
hotspot_data = pandas.read_csv('hotspot_historic.csv')
climate_data = pandas.read_csv('climate_historic.csv')

In [10]:
# Read data from CSV using pandas
hotspot_data = pandas.read_csv('hotspot_historic.csv')
climate_data = pandas.read_csv('climate_historic.csv')

collection_data = []

for index, row in climate_data.iterrows():
    document = {}

    # Format data appropriately for document object.
    document['date'] = datetime.strptime(row['date'], '%d/%m/%Y')
    document['station'] = int(row['station'])
    document["air_temperature_celcius"] = float(
        row["air_temperature_celcius"])
    document['relative_humidity'] = float(row['relative_humidity'])
    document['windspeed_knots'] = float(row['windspeed_knots'])
    document['max_wind_speed'] = float(row['max_wind_speed'])

    # Unncessary space at beginning of value is removed.
    # Also split precipation type and amount, to make it easier for sorting/searching later.
    precipitation = str(row['precipitation ']).replace(" ", "")
    document['precipitation_type'] = precipitation[-1]

    document['precipitation'] = float(precipitation[0:-1])
    document['ghi'] = float(row['GHI_w/m2'])

    # Get all hotspot records that match climate date & append to document.
    hotspot_date = document['date'].strftime("%-d/%m/%Y")
    
    hotspots = hotspot_data[hotspot_data['date'] == hotspot_date]

    # If there is a match, we clean up data & append records to document.
    if len(hotspots) > 0:
        document['hotspots'] = []
        for index, row in hotspots.iterrows():
            hotspot = {}
            hotspot['time'] = datetime.strptime(
                row['datetime'], '%Y-%m-%dT%H:%M:%S')
            hotspot['latitude'] = float(row['latitude'])
            hotspot['longitude'] = float(row['longitude'])
            hotspot['confidence'] = float(row['confidence'])
            hotspot['surface_temperature_celcius'] = float(
                row['surface_temperature_celcius'])
            document['hotspots'].append(hotspot)

    collection_data.append(document)
    
# Insert collection into database
collection.insert_many(collection_data)

In [11]:
result = collection.find_one({'date': datetime(2022, 12, 12)})
pprint(result)

{'_id': ObjectId('647f20bc02b5db9ca3359da8'),
 'air_temperature_celcius': 19.0,
 'date': datetime.datetime(2022, 12, 12, 0, 0),
 'ghi': 156.0,
 'hotspots': [{'confidence': 53.0,
               'latitude': -37.903,
               'longitude': 145.25,
               'surface_temperature_celcius': 44.0,
               'time': datetime.datetime(2022, 12, 12, 0, 45, 38)}],
 'max_wind_speed': 12.0,
 'precipitation': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


In [12]:
result = collection.aggregate([{'$unwind': '$hotspots'}, {'$match': {'hotspots.surface_temperature_celcius': {'$gte': 65, '$lte': 100}}}, {
    '$project': {'_id': 0, 'hotspots.longitude': 1, 'hotspots.latitude': 1, 'hotspots.surface_temperature_celcius': 1}}])

for each in result:
    pprint(each)

{'hotspots': {'latitude': -37.462,
              'longitude': 148.089,
              'surface_temperature_celcius': 90.0}}
{'hotspots': {'latitude': -37.46,
              'longitude': 148.102,
              'surface_temperature_celcius': 65.0}}
{'hotspots': {'latitude': -38.127,
              'longitude': 143.82,
              'surface_temperature_celcius': 77.0}}
{'hotspots': {'latitude': -37.294,
              'longitude': 141.232,
              'surface_temperature_celcius': 87.0}}
{'hotspots': {'latitude': -37.293,
              'longitude': 141.245,
              'surface_temperature_celcius': 85.0}}
{'hotspots': {'latitude': -37.247,
              'longitude': 141.278,
              'surface_temperature_celcius': 68.0}}
{'hotspots': {'latitude': -36.098,
              'longitude': 143.74,
              'surface_temperature_celcius': 67.0}}
{'hotspots': {'latitude': -37.331,
              'longitude': 143.122,
              'surface_temperature_celcius': 65.0}}
{'hotspots': {'lati

In [13]:
result = collection.find({'date': {'$in': [datetime(2022, 12, 15), datetime(2022, 12, 16)]}}, {
    '_id': 0, 'date': 1, 'hotspots.surface_temperature_celcius': 1, 'air_temperature_celcius': 1, 'relative_humidity': 1, 'max_wind_speed': 1})

for each in result:
    pprint(each)

{'air_temperature_celcius': 18.0,
 'date': datetime.datetime(2022, 12, 15, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 42.0},
              {'surface_temperature_celcius': 36.0},
              {'surface_temperature_celcius': 38.0},
              {'surface_temperature_celcius': 40.0}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18.0,
 'date': datetime.datetime(2022, 12, 16, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 43.0},
              {'surface_temperature_celcius': 33.0},
              {'surface_temperature_celcius': 54.0},
              {'surface_temperature_celcius': 73.0},
              {'surface_temperature_celcius': 55.0},
              {'surface_temperature_celcius': 75.0},
              {'surface_temperature_celcius': 55.0},
              {'surface_temperature_celcius': 66.0},
              {'surface_temperature_celcius': 56.0},
              {'surface_temperature_celcius': 60.0},
              {'surface_temperature_celc

In [14]:
result = collection.aggregate([{'$unwind': '$hotspots'}, {'$match': {'hotspots.confidence': {'$gte': 80, '$lte': 100}}}, {
    '$project': {'_id': 0, 'hotspots.time': 1, 'air_temperature_celcius': 1, 'hotspots.surface_temperature_celcius': 1, 'hotspots.confidence': 1}}])

for each in result:
    pprint(each)

{'air_temperature_celcius': 12.0,
 'hotspots': {'confidence': 100.0,
              'surface_temperature_celcius': 54.0,
              'time': datetime.datetime(2022, 10, 1, 15, 13, 56)}}
{'air_temperature_celcius': 12.0,
 'hotspots': {'confidence': 84.0,
              'surface_temperature_celcius': 52.0,
              'time': datetime.datetime(2022, 10, 1, 4, 10, 28)}}
{'air_temperature_celcius': 12.0,
 'hotspots': {'confidence': 89.0,
              'surface_temperature_celcius': 61.0,
              'time': datetime.datetime(2022, 10, 1, 4, 10, 28)}}
{'air_temperature_celcius': 11.0,
 'hotspots': {'confidence': 83.0,
              'surface_temperature_celcius': 50.0,
              'time': datetime.datetime(2022, 10, 2, 4, 53, 10)}}
{'air_temperature_celcius': 11.0,
 'hotspots': {'confidence': 85.0,
              'surface_temperature_celcius': 61.0,
              'time': datetime.datetime(2022, 10, 2, 4, 53, 9)}}
{'air_temperature_celcius': 13.0,
 'hotspots': {'confidence': 92.0,
      

In [15]:
result = collection.aggregate([{'$unwind': '$hotspots'}, {
    '$sort': {'hotspots.surface_temperature_celcius': -1}}, {'$limit': 10}])

for each in result:
    pprint(each)

{'_id': ObjectId('647f20bc02b5db9ca3359d87'),
 'air_temperature_celcius': 14.0,
 'date': datetime.datetime(2022, 11, 9, 0, 0),
 'ghi': 119.0,
 'hotspots': {'confidence': 100.0,
              'latitude': -37.477,
              'longitude': 143.352,
              'surface_temperature_celcius': 93.0,
              'time': datetime.datetime(2022, 11, 9, 4, 16, 48)},
 'max_wind_speed': 13.0,
 'precipitation': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 50.9,
 'station': 948702,
 'windspeed_knots': 5.9}
{'_id': ObjectId('647f20bc02b5db9ca3359d62'),
 'air_temperature_celcius': 13.0,
 'date': datetime.datetime(2022, 10, 3, 0, 0),
 'ghi': 115.0,
 'hotspots': {'confidence': 100.0,
              'latitude': -37.462,
              'longitude': 148.089,
              'surface_temperature_celcius': 90.0,
              'time': datetime.datetime(2022, 10, 3, 3, 58, 14)},
 'max_wind_speed': 12.0,
 'precipitation': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 46.2,
 'station': 94870

In [16]:
result = collection.aggregate(
    [{'$project': {'_id': 0, 'date': 1, 'numberOfFires': {'$size': {'$ifNull': ['$hotspots', []]}}}}])

for each in result:
    pprint({
        "date": each['date'].strftime("%d/%m/%Y"),
        "numberOfFires": each['numberOfFires']
    })

{'date': '31/12/2021', 'numberOfFires': 0}
{'date': '02/01/2022', 'numberOfFires': 0}
{'date': '03/01/2022', 'numberOfFires': 0}
{'date': '04/01/2022', 'numberOfFires': 0}
{'date': '05/01/2022', 'numberOfFires': 0}
{'date': '06/01/2022', 'numberOfFires': 0}
{'date': '07/01/2022', 'numberOfFires': 0}
{'date': '08/01/2022', 'numberOfFires': 0}
{'date': '09/01/2022', 'numberOfFires': 0}
{'date': '10/01/2022', 'numberOfFires': 0}
{'date': '11/01/2022', 'numberOfFires': 0}
{'date': '12/01/2022', 'numberOfFires': 0}
{'date': '13/01/2022', 'numberOfFires': 0}
{'date': '14/01/2022', 'numberOfFires': 0}
{'date': '15/01/2022', 'numberOfFires': 0}
{'date': '16/01/2022', 'numberOfFires': 0}
{'date': '17/01/2022', 'numberOfFires': 0}
{'date': '18/01/2022', 'numberOfFires': 0}
{'date': '19/01/2022', 'numberOfFires': 0}
{'date': '20/01/2022', 'numberOfFires': 0}
{'date': '21/01/2022', 'numberOfFires': 0}
{'date': '22/01/2022', 'numberOfFires': 0}
{'date': '23/01/2022', 'numberOfFires': 0}
{'date': '2

In [17]:
result = collection.aggregate(
    [{'$unwind': '$hotspots'}, {'$match': {'hotspots.confidence': {'$lt': 70}}}, {'$project': {'_id': 0, 'hotspots': 1}}])

for each in result:
    pprint(each)

{'hotspots': {'confidence': 62.0,
              'latitude': -37.463,
              'longitude': 148.109,
              'surface_temperature_celcius': 34.0,
              'time': datetime.datetime(2022, 10, 1, 15, 13, 56)}}
{'hotspots': {'confidence': 64.0,
              'latitude': -37.46,
              'longitude': 148.113,
              'surface_temperature_celcius': 52.0,
              'time': datetime.datetime(2022, 10, 1, 4, 10, 28)}}
{'hotspots': {'confidence': 50.0,
              'latitude': -37.466,
              'longitude': 148.1,
              'surface_temperature_celcius': 29.0,
              'time': datetime.datetime(2022, 10, 2, 23, 44, 31)}}
{'hotspots': {'confidence': 59.0,
              'latitude': -37.475,
              'longitude': 148.134,
              'surface_temperature_celcius': 40.0,
              'time': datetime.datetime(2022, 10, 2, 4, 53, 10)}}
{'hotspots': {'confidence': 68.0,
              'latitude': -37.456,
              'longitude': 148.11,
         

In [18]:
result = collection.aggregate([{'$unwind': '$hotspots'}, {'$group': {
    '_id': '$date', 'avgTemp': {'$avg': '$hotspots.surface_temperature_celcius'}}}])

for each in result:
    pprint({
        'date': each['_id'].strftime("%d/%m/%Y"),
        'avgTemp': each['avgTemp']
    })

{'avgTemp': 38.0, 'date': '23/10/2022'}
{'avgTemp': 59.0, 'date': '21/11/2022'}
{'avgTemp': 50.4, 'date': '27/10/2022'}
{'avgTemp': 58.25, 'date': '09/12/2022'}
{'avgTemp': 52.166666666666664, 'date': '18/10/2022'}
{'avgTemp': 46.25, 'date': '11/11/2022'}
{'avgTemp': 53.333333333333336, 'date': '10/10/2022'}
{'avgTemp': 36.0, 'date': '16/10/2022'}
{'avgTemp': 43.57142857142857, 'date': '02/10/2022'}
{'avgTemp': 58.8, 'date': '23/11/2022'}
{'avgTemp': 60.625, 'date': '29/11/2022'}
{'avgTemp': 52.0, 'date': '14/11/2022'}
{'avgTemp': 58.5, 'date': '05/11/2022'}
{'avgTemp': 47.0, 'date': '13/11/2022'}
{'avgTemp': 50.6, 'date': '08/12/2022'}
{'avgTemp': 70.0, 'date': '14/12/2022'}
{'avgTemp': 42.0, 'date': '28/11/2022'}
{'avgTemp': 57.8, 'date': '16/12/2022'}
{'avgTemp': 51.6, 'date': '17/10/2022'}
{'avgTemp': 62.75, 'date': '27/12/2022'}
{'avgTemp': 32.0, 'date': '24/12/2022'}
{'avgTemp': 53.0, 'date': '12/11/2022'}
{'avgTemp': 45.5, 'date': '08/11/2022'}
{'avgTemp': 61.3, 'date': '09/11/2

In [19]:
result = collection.aggregate([{'$sort': {'ghi': 1}}, {'$limit': 10}])

for each in result:
    pprint(each)

{'_id': ObjectId('647f20bc02b5db9ca3359d24'),
 'air_temperature_celcius': 5.0,
 'date': datetime.datetime(2022, 8, 2, 0, 0),
 'ghi': 47.0,
 'max_wind_speed': 5.1,
 'precipitation': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'_id': ObjectId('647f20bc02b5db9ca3359d03'),
 'air_temperature_celcius': 5.0,
 'date': datetime.datetime(2022, 6, 30, 0, 0),
 'ghi': 48.0,
 'max_wind_speed': 11.1,
 'precipitation': 0.0,
 'precipitation_type': 'A',
 'relative_humidity': 34.9,
 'station': 948701,
 'windspeed_knots': 4.3}
{'_id': ObjectId('647f20bc02b5db9ca3359d04'),
 'air_temperature_celcius': 5.0,
 'date': datetime.datetime(2022, 7, 1, 0, 0),
 'ghi': 49.0,
 'max_wind_speed': 14.0,
 'precipitation': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 33.1,
 'station': 948701,
 'windspeed_knots': 5.8}
{'_id': ObjectId('647f20bc02b5db9ca3359d0e'),
 'air_temperature_celcius': 6.0,
 'date': datetime.datetime(2022, 7, 11, 0, 0),
 'ghi': 56

In [20]:
result = collection.aggregate(
    [{'$match': {'precipitation_type': 'G', 'precipitation': {'$gte': 0.20, '$lte': 0.35}}}])

for each in result:
    pprint(each)

{'_id': ObjectId('647f20bc02b5db9ca3359c5b'),
 'air_temperature_celcius': 19.0,
 'date': datetime.datetime(2022, 1, 13, 0, 0),
 'ghi': 157.0,
 'max_wind_speed': 18.1,
 'precipitation': 0.31,
 'precipitation_type': 'G',
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'_id': ObjectId('647f20bc02b5db9ca3359ca6'),
 'air_temperature_celcius': 17.0,
 'date': datetime.datetime(2022, 3, 29, 0, 0),
 'ghi': 146.0,
 'max_wind_speed': 21.0,
 'precipitation': 0.24,
 'precipitation_type': 'G',
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'_id': ObjectId('647f20bc02b5db9ca3359cbc'),
 'air_temperature_celcius': 20.0,
 'date': datetime.datetime(2022, 4, 20, 0, 0),
 'ghi': 166.0,
 'max_wind_speed': 15.9,
 'precipitation': 0.31,
 'precipitation_type': 'G',
 'relative_humidity': 53.5,
 'station': 948701,
 'windspeed_knots': 7.2}
{'_id': ObjectId('647f20bc02b5db9ca3359cc2'),
 'air_temperature_celcius': 11.0,
 'date': datetime.datetime(2022, 4, 26, 0,

# question 3 



Single-field indexes:

date: It will be useful for tasks like finding climate data on a specific date, querying data for a range of dates, and finding the number of hotspot incidents each day. Thereby increasing efficiency of date based query execution involving sorting and date specific queries.

hotspots.confidence: the performance of queries that involve filtering by confidence values can be improved using index for confidence. It will be useful for tasks like finding records with a specific confidence range and finding records with confidence below a certain threshold.


Compound indexes:

"date": 1, "hotspots.surface_temperature_celcius": 1: This way it woul dbe beneficial for tasks that involve filtering data based on both date and surface temperature. It will optimize queries like finding surface temperature within a specific range on a particular date and finding the top records with the highest surface temperature.

Similarly multiple combinations of date with other factors like wind speed and air temperature, relative humidity for achieving better performance for query based on climate attributes involving date based search.


Using MongoDB's sharding feature to distribute data across multiple servers, allowing for horizontal scalability and increased write throughput.